In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import calendar
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("Raw_Dataset\Customer_comment.csv")

In [ ]:
df.info()

This code performs sentiment analysis on airline customer reviews to understand passengers’ overall emotions and opinions. It calculates a sentiment score for each review using VADER and categorizes them as positive, neutral, or negative. The results are then visualized in a bar chart showing how reviews are distributed across these categories. This analysis helps identify the general satisfaction level of travelers and sets the stage for deeper insights, such as exploring what factors contribute most to negative or positive experiences.

In [ ]:
sia = SentimentIntensityAnalyzer()

def sentence_level_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.0
    sentences = sent_tokenize(text)
    scores = [sia.polarity_scores(s)['compound'] for s in sentences]
    return sum(scores) / len(scores)

df['Sentiment_Score'] = df['Review_content'].apply(sentence_level_sentiment)

def label_sentiment(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['Sentiment_Analysis'] = df['Sentiment_Score'].apply(label_sentiment)

sns.countplot(x='Sentiment_Analysis', data=df, order=['Positive', 'Neutral', 'Negative'], palette='coolwarm')
plt.title('Sentiment Distribution of Airline Reviews')
plt.show()

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np

geolocator = Nominatim(user_agent="airline_data_cleaning")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

def get_location(lat, lon):
    try:
        if pd.notna(lat) and pd.notna(lon):
            location = reverse((lat, lon), language='en')
            if location and location.address:
                addr = location.raw.get('address', {})
                city = addr.get('city') or addr.get('town') or addr.get('village') or addr.get('state')
                return city if city else location.address
    except:
        return np.nan
    return np.nan

# Fill missing Start_Location
mask_start = df['Start_Location'].isna() & df['Start_Latitude'].notna() & df['Start_Longitude'].notna()
df.loc[mask_start, 'Start_Location'] = df.loc[mask_start].apply(
    lambda row: get_location(row['Start_Latitude'], row['Start_Longitude']), axis=1
)

# Fill missing End_Location
mask_end = df['End_Location'].isna() & df['End_Latitude'].notna() & df['End_Longitude'].notna()
df.loc[mask_end, 'End_Location'] = df.loc[mask_end].apply(
    lambda row: get_location(row['End_Latitude'], row['End_Longitude']), axis=1
)

print("✅ Missing Start and End locations filled (where coordinates available).")


This script automatically fills missing latitude and longitude values for both Start_Location and End_Location in the airline reviews dataset.

🔍 How It Works

Cleans location names using the clean_location_name() function — removing unnecessary words like "airport", punctuation, and extra spaces.

Uses the Nominatim API (via geopy) to geocode cleaned location names and retrieve accurate latitude and longitude values.

Iterates through missing entries in both start and end coordinates:

If valid coordinates are found, they are added to the dataset.

If not, a warning is printed.

Reports progress and summary of remaining missing values.

✅ This step ensures complete and consistent geolocation data for all routes, improving accuracy for mapping and spatial visualizations.

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re
import numpy as np
import pandas as pd

geolocator = Nominatim(user_agent="airline_location_filler")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def clean_location_name(name):
    if not isinstance(name, str):
        return np.nan
    name = name.lower()
    name = re.sub(r'\([^)]*\)', '', name)  # remove text in parentheses
    name = re.sub(r'\b(international|airport|airpt|airlines?|terminal|int|intl)\b', '', name)
    name = re.sub(r'[-,]', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    return name.strip().title()

def get_lat_lon(location):
    try:
        loc = geocode(location)
        if loc:
            return loc.latitude, loc.longitude
    except:
        return np.nan, np.nan
    return np.nan, np.nan

# ---------- Fill missing START coordinates ----------
missing_start = df[df['Start_Latitude'].isna() | df['Start_Longitude'].isna()]

for idx, row in missing_start.iterrows():
    cleaned = clean_location_name(row['Start_Location'])
    if isinstance(cleaned, str) and cleaned != "":
        lat, lon = get_lat_lon(cleaned)
        if pd.notna(lat) and pd.notna(lon):
            df.at[idx, 'Start_Latitude'] = lat
            df.at[idx, 'Start_Longitude'] = lon
            print(f"✅ Start filled: {row['Start_Location']} → ({lat:.4f}, {lon:.4f})")
        else:
            print(f"⚠️ Start not found: {row['Start_Location']}")

# ---------- Fill missing END coordinates ----------
missing_end = df[df['End_Latitude'].isna() | df['End_Longitude'].isna()]

for idx, row in missing_end.iterrows():
    cleaned = clean_location_name(row['End_Location'])
    if isinstance(cleaned, str) and cleaned != "":
        lat, lon = get_lat_lon(cleaned)
        if pd.notna(lat) and pd.notna(lon):
            df.at[idx, 'End_Latitude'] = lat
            df.at[idx, 'End_Longitude'] = lon
            print(f"✅ End filled: {row['End_Location']} → ({lat:.4f}, {lon:.4f})")
        else:
            print(f"⚠️ End not found: {row['End_Location']}")

print("\n✅ All missing Start and End coordinates processed.")
print("Remaining missing values:")
print("Start_Latitude:", df['Start_Latitude'].isna().sum())
print("End_Latitude:", df['End_Latitude'].isna().sum())


This script cleans and standardizes location names and fills in missing route information for the airline dataset.

⚙️ Step-by-Step Summary

clean_location_name() function:

Converts names to lowercase.

Removes unnecessary words like “airport”, “international”, etc.

Removes text inside parentheses and extra punctuation/spaces.

Returns a clean, properly capitalized location name.

Cleans both location columns:

Applies the cleaning function to Start_Location and End_Location.

Fills missing routes:

For any missing Route, constructs it as "Start_Location to End_Location".

✅ This ensures all routes are consistently formatted and complete, making the dataset ready for visualization and analysis.

In [ ]:
import pandas as pd
import numpy as np
import re

def clean_location_name(name):
    if not isinstance(name, str):
        return np.nan
    name = name.lower()
    name = re.sub(r'\([^)]*\)', '', name)  # remove text in parentheses
    name = re.sub(r'\b(international|airport|airpt|airlines?|terminal|int|intl)\b', '', name)
    name = re.sub(r'[-,]', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    return name.strip().title()

# --- Clean Start and End locations ---
df['Start_Location'] = df['Start_Location'].apply(clean_location_name)
df['End_Location'] = df['End_Location'].apply(clean_location_name)

# --- Fill missing Route values ---
missing_routes = df['Route'].isna()
df.loc[missing_routes, 'Route'] = df.loc[missing_routes].apply(
    lambda row: f"{row['Start_Location']} to {row['End_Location']}"
    if pd.notna(row['Start_Location']) and pd.notna(row['End_Location'])
    else np.nan,
    axis=1
)


We removed Layover_Route because it was:

🧩 Redundant — the main Route column (constructed as Start → End) already represents the flight path. The layover doesn’t add unique analytical value for most visualizations or sentiment insights.

📉 Incomplete — only a small fraction of entries (484 out of 3575) had Layover_Route filled, making it too sparse to be useful for modeling or aggregation.

🧼 Simplifies the dataset — removing it makes the data cleaner and more consistent, focusing on core fields (Start_Location, End_Location, and Route) that are now complete and standardized.

In short: it was mostly empty, repetitive, and unnecessary for your current analysis goals.

In [ ]:
df = df.drop(columns=['Layover_Route'])
df = df.drop(columns=['Flying_Date'])
print("✅ Layover_Route removed successfully.")

Perfect — that line saves your fully cleaned and standardized dataset to a CSV file named cleaned_airline_reviews.csv in your working directory.

Here’s what it does:

index=False → prevents pandas from adding row numbers as an extra column.

encoding='utf-8-sig' → ensures correct encoding (especially for non-English text like Arabic city names).

to_csv() → writes the DataFrame to a CSV file that you can easily open in Excel, Power BI, or reload later for further analysis.

In short:

💾 You’ve successfully exported your final cleaned dataset — ready for analysis, visualization, or machine learning.

In [ ]:
df.to_csv("cleaned_airline_reviews.csv", index=False, encoding='utf-8-sig')
print("✅ Data exported to 'cleaned_airline_reviews.csv'")